# Market Mix Modelling for an e-commerce firm

#### Problem statment:

Eleckart, an e-commerce firm based out of Ontario, Canada spends a significant amount of money on marketing. The marketing budget is spent on commercials, online campaigns and pricing & promotion strategies. Eleckart’s CFO feels that the money spent over the last 12 months did not have enough impact and that the budget can be cut or optimally reallocated across marketing levers to improve revenue.

#### Goal:
As part of the marketing team, which is working on budget optimisation, we need to develop a market mix model with the following objectives:

- Observe the actual impact of different marketing variables over the last one year
- Recommend the optimal budget allocation for different marketing levers for the next year


In [1]:
# import libraties
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Phase 1: Understanding the data

In [2]:
products = pd.read_csv('ConsumerElectronics.csv')

In [3]:
products.head()

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
0,ACCCX3S58G7B5F6P,2015-10-17 15:11:54,2015,10,3.419301e+15,3.419301e+15,6400,1,\N,\N,COD,5,-1.01299130778588E+018,-7.79175582905735E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
1,ACCCX3S58G7B5F6P,2015-10-19 10:07:22,2015,10,1.420831e+15,1.420831e+15,6900,1,\N,\N,COD,7,-8.99032457905512E+018,7.33541149097431E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
2,ACCCX3S5AHMF55FV,2015-10-20 15:45:56,2015,10,2.421913e+15,2.421913e+15,1990,1,\N,\N,COD,10,-1.0404429420466E+018,-7.47768776228657E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
3,ACCCX3S5AHMF55FV,2015-10-14 12:05:15,2015,10,4.416592e+15,4.416592e+15,1690,1,\N,\N,Prepaid,4,-7.60496084352714E+018,-5.83593163877661E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
4,ACCCX3S5AHMF55FV,2015-10-17 21:25:03,2015,10,4.419525e+15,4.419525e+15,1618,1,\N,\N,Prepaid,6,2.8945572083453E+018,5.34735360997242E+017,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3


## 1.2 Understanding Media Investment (Krishnan)

In [4]:
from pandas import ExcelWriter
from pandas import ExcelFile

media_invest = pd.read_excel('Media data and other information.xlsx', sheet_name='Media Investment',skiprows=2)

print(media_invest.columns)

Index(['Unnamed: 0', 'Year', 'Month', 'Total Investment', 'TV', 'Digital',
       'Sponsorship', 'Content Marketing', 'Online marketing', ' Affiliates',
       'SEM', 'Radio', 'Other'],
      dtype='object')


In [5]:
media_invest.head()

,Unnamed: 0,Year,Month,Total Investment,TV,Digital,Sponsorship,Content Marketing,Online marketing,Affiliates,SEM,Radio,Other
0,NaN,2015.0,7.0,17.061775,0.215330,2.533014,7.414270,0.000933,1.327278,0.547254,5.023697,NaN,NaN
1,NaN,2015.0,8.0,5.064306,0.006438,1.278074,1.063332,0.000006,0.129244,0.073684,2.513528,NaN,NaN
2,NaN,2015.0,9.0,96.254380,3.879504,1.356528,62.787651,0.610292,16.379990,5.038266,6.202149,NaN,NaN
3,NaN,2015.0,10.0,170.156297,6.144711,12.622480,84.672532,3.444075,24.371778,6.973711,31.927011,NaN,NaN
4,NaN,2015.0,11.0,51.216220,4.220630,1.275469,14.172116,0.168633,19.561574,6.595767,5.222032,NaN,NaN


In [6]:
#Dropping the Unnamed column
media_invest.drop(media_invest.columns[[0]],axis=1,inplace=True)

In [7]:
media_invest.head()

,Year,Month,Total Investment,TV,Digital,Sponsorship,Content Marketing,Online marketing,Affiliates,SEM,Radio,Other
0,2015.0,7.0,17.061775,0.215330,2.533014,7.414270,0.000933,1.327278,0.547254,5.023697,NaN,NaN
1,2015.0,8.0,5.064306,0.006438,1.278074,1.063332,0.000006,0.129244,0.073684,2.513528,NaN,NaN
2,2015.0,9.0,96.254380,3.879504,1.356528,62.787651,0.610292,16.379990,5.038266,6.202149,NaN,NaN
3,2015.0,10.0,170.156297,6.144711,12.622480,84.672532,3.444075,24.371778,6.973711,31.927011,NaN,NaN
4,2015.0,11.0,51.216220,4.220630,1.275469,14.172116,0.168633,19.561574,6.595767,5.222032,NaN,NaN


In [8]:
#Retaining only the rows corresponding to the 12 months from Jul2015 to Jun2016
media_invest.drop(media_invest[media_invest.index > 11].index,inplace=True)

In [9]:
#Replacing all the missing values with zeros
media_invest.fillna(0,inplace=True)

In [10]:
#Converting the year and month values to integer values from floating points
media_invest = media_invest.astype({"Year":'int64', "Month":'int64'}) 

In [11]:
#Trimming the columns of any white spaces
media_invest.columns = media_invest.columns.str.strip()

In [12]:
media_invest

,Year,Month,Total Investment,TV,Digital,Sponsorship,Content Marketing,Online marketing,Affiliates,SEM,Radio,Other
0,2015,7,17.061775,0.215330,2.533014,7.414270,0.000933,1.327278,0.547254,5.023697,0.00,0.000000
1,2015,8,5.064306,0.006438,1.278074,1.063332,0.000006,0.129244,0.073684,2.513528,0.00,0.000000
2,2015,9,96.254380,3.879504,1.356528,62.787651,0.610292,16.379990,5.038266,6.202149,0.00,0.000000
3,2015,10,170.156297,6.144711,12.622480,84.672532,3.444075,24.371778,6.973711,31.927011,0.00,0.000000
4,2015,11,51.216220,4.220630,1.275469,14.172116,0.168633,19.561574,6.595767,5.222032,0.00,0.000000
5,2015,12,106.745312,5.397502,3.063360,56.705419,1.067307,22.503756,6.826938,11.181030,0.00,0.000000
6,2016,1,74.196000,4.380000,0.456000,4.200000,0.900000,22.900000,7.370000,4.200000,2.70,27.090000
7,2016,2,48.052000,2.586008,1.912651,11.725555,0.596771,19.892366,6.465019,4.873630,0.00,0.000000
8,2016,3,100.024621,9.300000,2.100000,41.590000,0.370000,18.437818,6.217667,5.193591,0.87,15.945545
9,2016,4,56.848241,5.205196,0.871892,24.308354,0.034876,16.514510,5.693456,4.219958,0.00,0.000000


In [13]:
media_invest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 12 columns):
Year                 12 non-null int64
Month                12 non-null int64
Total Investment     12 non-null float64
TV                   12 non-null float64
Digital              12 non-null float64
Sponsorship          12 non-null float64
Content Marketing    12 non-null float64
Online marketing     12 non-null float64
Affiliates           12 non-null float64
SEM                  12 non-null float64
Radio                12 non-null float64
Other                12 non-null float64
dtypes: float64(10), int64(2)
memory usage: 1.2 KB


#### Observations:
- media_invest dataframe has 12 columns
- `Year` and `month` columns represent the year and month of investment
- The other 10 columns represent the monthly investments in 10 different advertising channels during the period Jul2015-Jun2016 



## 1.3 Understanding monthly NPS score

In [14]:
nps_score = pd.read_excel('Media data and other information.xlsx', sheet_name='Monthly NPS Score')
print(nps_score.columns)

Index(['Unnamed: 0', 'July'15', 'Aug'15', 'Sept'15', 'Oct'15', 'Nov'15',
       'Dec'15', 'Jan'16', 'Feb'16', 'Mar'16', 'Apr'16', 'May'16', 'June'16'],
      dtype='object')


In [15]:
nps_score = nps_score.T

In [16]:
nps_score.columns = ['NPS','Stock Index']

In [17]:
nps_score = nps_score.iloc[1:]

In [18]:
nps_score

,NPS,Stock Index
July'15,54.5996,1177
Aug'15,59.9871,1206
Sept'15,46.9254,1101
Oct'15,44.3984,1210
Nov'15,47,1233
Dec'15,45.8,1038
Jan'16,47.093,1052
Feb'16,50.3274,1222
Mar'16,49.0206,1015
Apr'16,51.8276,1242


In [19]:
nps_score.reset_index(level=0, inplace=True)

In [20]:
#Replacing the long month names with the shorter month names to help with the datetime parsing
nps_score = nps_score.replace('July', 'Jul', regex=True)
nps_score = nps_score.replace('Sept', 'Sep', regex=True)
nps_score = nps_score.replace('June', 'Jun', regex=True)

In [21]:
nps_score['MonthYear'] =  pd.to_datetime(nps_score['index'], format="%b'%y")

In [22]:
nps_score.drop(['index'],axis=1,inplace=True)

In [23]:
nps_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
NPS            12 non-null float64
Stock Index    12 non-null float64
MonthYear      12 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 368.0 bytes


In [24]:
nps_score

,NPS,Stock Index,MonthYear
0,54.599588,1177.0,2015-07-01
1,59.987101,1206.0,2015-08-01
2,46.925419,1101.0,2015-09-01
3,44.398389,1210.0,2015-10-01
4,47.000000,1233.0,2015-11-01
5,45.800000,1038.0,2015-12-01
6,47.093031,1052.0,2016-01-01
7,50.327406,1222.0,2016-02-01
8,49.020550,1015.0,2016-03-01
9,51.827605,1242.0,2016-04-01


#### Observations:
- nps_score dataframe has 3 columns
- `NPS` column represents the NPS score for the corresponding month value reprsented by `MonthYear` column
- `Stock Index` column represents the company stock price for the corresponding month value reprsented by `MonthYear` column

## 1.4 Understanding Special Sale Calendar (Krishnan)

In [147]:
spl_sales = pd.read_excel('Media data and other information.xlsx', sheet_name='Special Sale Calendar')
spl_sales.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Sales Calendar'], dtype='object')

In [148]:
#Dropping the first column 
spl_sales.drop(['Unnamed: 0'],axis=1,inplace=True)

In [149]:
#Renaming the column to Year column as it contains year values
spl_sales.rename(columns={"Unnamed: 1": "Year"},inplace=True)

#Replacing all the missing values with zeros
spl_sales['Year'].fillna(0,inplace=True)

In [150]:
#Retaining only the rows corresponding to the special sale dates
spl_sales.drop(spl_sales[spl_sales.index > 11].index,inplace=True)

In [151]:
#Converting the year and month values to integer values from floating points
spl_sales = spl_sales.astype({"Year":'int64'}) 

In [153]:
#Filling up all the Year column values with the appropriate values
spl_sales.at[(spl_sales.index >= 1) & (spl_sales.index<=5),'Year']=2015
spl_sales.at[(spl_sales.index >= 7) & (spl_sales.index<=11),'Year']=2016

In [154]:
#Creating a column with the row index values
spl_sales['rowIndex'] = spl_sales.index

In [155]:
import datetime

#Custom function for extracting the start and end dates of the sale
def extractSaleDates(index, daterange):
    
    #Removing any trailing/leading white spaces
    datestr = daterange.strip()
    
    if(index != 5):
        
        if(index < 5):
            year=2015
        else:
            year=2016
    
        #Replacing 'th' with empty spaces
        datestr = datestr.replace("th","")
   
        #Splitting into day and month parts
        datestr = datestr.split(" ")
    
        days = datestr[0][1:].split("-")
        month = datestr[1][0:3]
    
        #Converting month name to a number
        monthNo = datetime.datetime.strptime(month, '%b').month
        startDay = int(days[0])
        endDay = int(days[1])
          
        startDate = datetime.datetime(year=year, month=monthNo, day=startDay)
        endDate = datetime.datetime(year=year, month=monthNo, day=endDay)
    
        if(startDate < endDate):    
            dateinfo = [startDate,endDate]
        else:
            dateinfo = [endDate,startDate]
            
    else:
        # Special case (25th Dec'15 - 3rd Jan'16)
        
        #Replacing 'th' with empty spaces
        datestr = datestr.replace("th","")
        datestr = datestr.replace("rd","")
        
        
        #Splitting the string into two parts
        datestr = datestr.split("-")
        startDatestr = datestr[0].replace(" ","")[1:]
        endDatestr = datestr[1].replace(" ","")[:-1]
        
        startDate = datetime.datetime.strptime(startDatestr, "%d%b'%y")
        endDate = datetime.datetime.strptime(endDatestr, "%d%b'%y")
        
        
        dateinfo = [startDate,endDate]
           
               
    return dateinfo
    

In [156]:
#Creating the "Start Date" and "End Date" columns to represent the start and end dates of the special sale window
spl_sales['Start Date'] = spl_sales.apply(lambda x: extractSaleDates(x['rowIndex'],x['Sales Calendar'])[0],axis=1)
spl_sales['End Date'] = spl_sales.apply(lambda x: extractSaleDates(x['rowIndex'],x['Sales Calendar'])[1],axis=1)

In [157]:
#Dropping the columns that are no longer required
spl_sales.drop(['rowIndex'],axis=1,inplace=True)
spl_sales.drop(['Year'],axis=1,inplace=True)
spl_sales.drop(['Sales Calendar'],axis=1,inplace=True)

In [158]:
spl_sales

,Start Date,End Date
0,2015-07-18,2015-07-19
1,2015-08-15,2015-08-17
2,2015-08-28,2015-08-30
3,2015-10-15,2015-10-17
4,2015-11-07,2015-11-14
5,2015-12-25,2016-01-03
6,2016-01-20,2016-01-22
7,2016-02-01,2016-02-02
8,2016-02-20,2016-02-21
9,2016-02-14,2016-02-15


#### Observations:
- `spl_sales` dataframe contains the special sale calendar between the period July 2015 - Jun 2016
- `spl_sales` dataframe contains two columns `Start Date` and `End Date` that contain the start and end dates for each special sale